# Lab 5 Motor Feedback

The purpose of this project is to develop a closed-loop PID program using the Pololu Zumo robot. We were provided an example program from Dr. Paul Talaga which used motor encoder data in a PID loop to move the robot a specific distance or to turn a certain amount. The goal for the project is to implement a PID loop using either a different sensor or to significantly improve Dr. Talaga's PID program.

My first goal was to attempt understanding the example PID program, though without developing or working with it for very long it was difficult to interpret in the time remaining for the project. My hope was to use the IMU's magnetometer and use a compass reading as an input to the PID loop. The Zumo library comes with a Rotation Resist example, which uses gyroscope data as the input to the PID loop. After a short calibration time, the robot resists attempts to rotate it by turning the motors against the direction of the turn. 

In order to repurpose the code for use with the previous project's I2C control, the turnAngle value can be modified in the main loop which forces the bot to turn a defined number of degrees from its previous orientation. [This code](RotationResist/RotationResist.ino) uses the IMU built into the Zumo robot to resist turns. I modified it to shift the target orientation by 90 degrees every few seconds. This could be used with the previous I2C program to command precise turns or to maintain a heading while the zumo moves a certain distance

This [demo](Media/ZumoRotationResistStart.mp4) shows the calibration of the robot at the beginning of the program. This [demo](Media/ZumoRotationResist.mp4) shows the Zumo bot resisting turns while also performing 90 degree turns on intervals.

Further work with the gyro PID would include adding the loop within the distance and turn loop of Dr. Talaga's PID example. The inner loop would use the gyro data to maintain orientation, while the outer loop could use encoder data to control the distance moved. 

This project has applications in ethical considerations as an engineer as PID loops can have major consequences in larger projects. An unstable PID loop can present danger to users or bystanders, even with the Zumo robots. The Zumo robots have pinch points and run the risk of driving away unpredictably if the PID loop becomes unstable. I can make sure my program is fail-safe when unexpected errors occur.

# Appendix: Code

Rotation Resist example, modified for 90 degree turns repeatedly
       
       
        #include <Wire.h>
        #include <Zumo32U4.h>

        // This is the maximum speed the motors will be allowed to turn.
        // A maxSpeed of 400 lets the motors go at top speed.  Decrease
        // this value to impose a speed limit.
        const int16_t maxSpeed = 400;

        // Change next line to this if you are using the older Zumo 32U4
        // with a black and green LCD display:
        Zumo32U4LCD display;
        //Zumo32U4OLED display;

        Zumo32U4ButtonA buttonA;
        Zumo32U4Motors motors;
        Zumo32U4IMU imu;

        #include "TurnSensor.h"
        // track the time between turns
        uint16_t delays = 0;
        // interval between turns, milliseconds
        uint16_t interval = 3000;

        void setup()
        {
        turnSensorSetup();
        delay(500);
        turnSensorReset();

        display.clear();
        display.print(F("Try to"));
        display.gotoXY(0, 1);
        display.print(F("turn me!"));
        }

        void loop()
        {
        // Read the gyro to update turnAngle, the estimation of how far
        // the robot has turned, and turnRate, the estimation of how
        // fast it is turning.
        turnSensorUpdate();
        if((millis() - delays) > interval){
            turnAngle += turnAngle90;
            delays = millis();
        }
        // Calculate the motor turn speed using proportional and
        // derivative PID terms.  Here we are a using a proportional
        // constant of 56 and a derivative constant of 1/20.
        int32_t turnSpeed = -(int32_t)turnAngle / (turnAngle1 / 56)
            - turnRate / 20;

        // Constrain our motor speeds to be between
        // -maxSpeed and maxSpeed.
        turnSpeed = constrain(turnSpeed, -maxSpeed, maxSpeed);
        motors.setSpeeds(-turnSpeed, turnSpeed);
        display.gotoXY(0, 0);
        display.print(turnAngle);

        }